In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import yfinance as yf
from scipy.stats import norm

In [3]:
class Binomial_Tree():
    """
    Binary_tree class.
    Inputs:
        -   S: stock price
        -   r: risk-free interest rate
        -   vol: volatility % in decimals
        -   T: Time period
        -   N: Number of steps/intervals
        -   auto: Compute tree automatically, True as default
    """

    def __init__(self,S,r,vol,T,N, K,auto = True):
        
        self.S = S
        self.r = r
        self.vol = vol
        self.T = T
        self.N = N
        self.dt = T/N
        
        if auto == True:
            self.build_tree()
            self.valueOptionMatrix(K)

    def build_tree(self):
    
        matrix = np.zeros((self.N+1,self.N+1))
        
        u = np.exp(self.vol*np.sqrt(self.dt))
        d = np.exp(-self.vol*np.sqrt(self.dt))
        matrix[0,0] = self.S
        
        
        for i in np.arange(self.N+1) :
            for j in np.arange(i+1) : 

                matrix[i,j] = self.S*(u**(j))*(d**(i-j))
        
        self.tree = matrix

    def valueOptionMatrix(self,K):

        self.K = K

        columns = self.tree.shape[1]
        rows = self.tree.shape[0]
        v_tree_european = np.copy(self.tree)
        v_tree_european_put = np.copy(self.tree)
        v_tree_american = np.copy(self.tree)
        v_tree_american_put = np.copy(self.tree)


        u= np.exp(self.vol*np.sqrt(self.dt))

        d= np.exp(-self.vol*np.sqrt(self.dt))

        p= (np.exp(self.r*self.dt) - d)/(u-d)   

        for c in np.arange(columns):
            St = self.tree[rows - 1, c] 
            v_tree_european[rows - 1, c] = max(0.,  St - self.K)
            v_tree_european_put[rows -1,c] = max(0., self.K - St)
            v_tree_american[rows -1,c] = max(0., St-self.K)
            v_tree_american_put[rows -1,c] = max(0., self.K - St)

        for i in np.arange(rows - 1)[:: -1]:
            for j in np.arange(i + 1):
                european_down = v_tree_european[ i + 1, j ]
                european_up = v_tree_european[ i + 1, j + 1]

                european_down_put = v_tree_european_put[ i + 1, j ]
                european_up_put = v_tree_european_put[ i + 1, j + 1]

                american_down= v_tree_american[ i + 1, j ]
                american_up= v_tree_american[ i + 1, j + 1]

                american_down_put = v_tree_american_put[i + 1, j ]
                american_up_put = v_tree_american_put[i + 1, j +1]

                
                v_tree_european[i , j ] = np.exp(-self.r*self.dt)*(p*european_up + (1-p)*european_down)

                v_tree_european_put[i , j ] = np.exp(-self.r*self.dt)*(p*european_up_put + (1-p)*european_down_put)
                
                v_tree_american[i,j] = max(np.exp(-self.r*self.dt)*(p*american_up + (1-p)*american_down), self.tree[i,j]-self.K)

                v_tree_american_put[i,j] = max(np.exp(-self.r*self.dt)*(p*american_up_put + (1-p)*american_down_put), self.K - self.tree[i,j])
       
        self.v_tree_european = v_tree_european
        self.v_tree_european_put = v_tree_european_put
        self.v_tree_american = v_tree_american
        self.v_tree_american_put = v_tree_american_put
        self.delta = (v_tree_european[1,1] - v_tree_european[1,0])/(self.S*u - self.S*d)

        


In [ ]:
"""
Black scholes model
"""

import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import norm

class Black_scholes():
    """
    Black_scholes model class.
    Inputs:
        -   S: stock price
        -   r: risk-free interest rate
        -   vol: volatility fo the stock % in decimals
        -   T: Time period
        -   N: Number of steps/intervals
        -   K: Strike price
        -   auto: Compute euler,exact method and values. True as default
    """

    def __init__(self, S,r,vol, T,N, K,auto = True):

        self.S = S
        self.r = r
        self.vol = vol
        self.T = T
        self.N = N
        self.dt = T/N
        self.K = K
        self.taos = self.T - np.arange(0,self.N+1)*self.dt

        self.delta = None
        self.deltas = None
        self.cash = None

        if auto == True:
            self.option_values(mode = "euler")
            self.option_values(mode = "exact")


    def exact_method(self):
        """
        Stocks price of each interval N in period T 
        using the exact solution of Black scholes
        """
        ex_St= np.zeros(self.N+1)
        ex_St[0] = self.S

        #### Begin Pre-computations
        pre_1 = (self.r-(1/2)*self.vol**2)*self.dt
        pre_2 = self.vol*np.sqrt(self.dt)
        ###### End Pre-computations

        for m in range(1,self.N+1):
            Z_m = np.random.normal(0,1,1)
            ex_St[m] = ex_St[m-1]*np.exp(pre_1 + pre_2*(Z_m))
            S_t = ex_St[m]
        
        self.ex_St = ex_St


    def euler_method(self):
        """
        Stocks price of each interval N in period T 
        using the euler approximation solution of Black scholes
        """

        eu_St = np.zeros(self.N+1)
        eu_St[0] = self.S

        #### Begin Pre-computations        
        pre_1 = self.r*self.dt
        pre_2 = self.vol*np.sqrt(self.dt)
        #### End Pre-computations

        for m in range(1,self.N+1):
            Z_m = np.random.normal(0,1,1)
            eu_St[m] = eu_St[m-1] + eu_St[m-1]*pre_1+ eu_St[m-1]*pre_2*Z_m
        self.eu_St = eu_St
    
    def option_values(self,mode = "exact"):
        """
        Expected value of an European price call option written on an asset in the Black-scholes model
        And hedge parameter
    
        Inputs:
            - mode = If use the exact or euler method of stock price
        """
        
        if mode == "euler":
            if not hasattr(self,'eu_St'):
                self.euler_method()
            self.eu_Vt = self.option_price(mode)

        elif mode == "exact":
            if not hasattr(self, mode):
                self.exact_method()

            self.ex_Vt = self.option_price(mode)

            ## Hedge parameter for part II question 4
            self.delta = norm.cdf((np.log(self.ex_St[0]/self.K) +  (self.r + 0.5*(self.vol**2))*(self.T))/(self.vol*np.sqrt(self.T)))

    
    def option_price(self,mode, vol_hedge = None):
        """
        Computes the expected price at time t of an european call option
        Inputs:
            - m: Position in time
            - St: Stock value
            . vol_hedge: Volatility of hedge, set as stock volatility as default
        """
        if mode == "exact":
            St = self.ex_St
        if mode == "euler":
            St = self.eu_St
        if vol_hedge == None:
            vol_hedge = self.vol

        d1s = (np.log(St/self.K) + (self.r + 0.5*(vol_hedge**2)*self.taos))/(self.vol*np.sqrt(self.taos))
        d2s = d1s - vol_hedge*np.sqrt(self.taos)
        Vt = St*norm.cdf(d1s) - np.exp(-self.r*self.taos)*self.K*norm.cdf(d2s)

        return Vt



In [2]:
aapl = yf.Ticker("AAPL")
n_years = 1
# get historical data for AAPL stock for n years
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=365*n_years)
aapl_data = yf.download('AAPL', start=start_date, end=end_date, progress=False)
option_expiry = '2024-09-20'

opt = aapl.option_chain(option_expiry)
opt.puts['ExpirationDate'] = option_expiry
puts_df = opt.puts.copy(deep=True)
puts_df.to_csv('AAPL_puts.csv')
puts_df = puts_df[puts_df['inTheMoney'] == True]
puts_df.sort_values('strike', inplace=True)
puts_df.reset_index(drop=True, inplace=True)

print(f'Last Traded Price of put option with Expiry: {puts_df.loc[0,"ExpirationDate"]}, Strike price: ${puts_df.loc[0,"strike"]}, and Open Interest: {puts_df.loc[0,"openInterest"]} is ${puts_df.loc[0,"lastPrice"]}')


Last Traded Price of put option with Expiry: 2024-09-20, Strike price: $175.0, and Open Interest: 30243 is $11.28


## Question 2